In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', None)

jeopardy = pd.read_csv('jeopardy.csv')
jeopardy.columns = ['show_number', 'air_date', 'round', 'category', 'value', 'question', 'answer']
jeopardy['value'] = jeopardy['value'].map(lambda v: int(v[1:].replace(',','')) if v[1].isdigit() else 0)

game_jeopardy = jeopardy[(~jeopardy.isnull().any(axis=1)) & (~jeopardy.question.str.contains(pat='<a'))]
# game_jeopardy

In [ ]:
def init_board(board):
    categories = board.category.unique()
    board_data = {}
    question_data = {}
    questions_remaining = 0
    for cat in categories:
        questions = board[board.category == cat]
        questions.sort_values(by='value')
        board_data[cat] = []
        question_data[cat] = []
        for i in range(5):
            if i > len(questions) - 1:
                board_data[cat].append(0)
                question_data[cat].append((None, None))
            else:
                row = questions.iloc[i]
                value, q, a = row.value, row.question, row.answer
                board_data[cat].append(value)
                question_data[cat].append((q, a))
                questions_remaining += 1
    board_df = pd.DataFrame.from_dict(board_data)
    question_df = pd.DataFrame.from_dict(question_data)
    return board_df, question_df, questions_remaining

In [ ]:
from random import randint
from IPython.display import display, HTML

def play_game():
    
    while True:
        game_round = 1
        
        # Set Player Number
        number_of_players = 0
        while True:
            number_of_players = input("How many players? ")
            if number_of_players.isdigit():
                break
        player_score = []
        for i in range(int(number_of_players)):
            player_score.append([f'Player {i + 1}', 0])
            
        # Randomly pick show
        total_show_numbers = len(game_jeopardy["show_number"].unique())
        random_show_number = randint(1, total_show_numbers-1)
        showNumber = game_jeopardy.iloc[random_show_number].show_number
        print('Show Number: ' + str(showNumber) + '\n')
        show = game_jeopardy[game_jeopardy.show_number == showNumber]
        
        # Play Each Round
        while game_round < 3:
            r = "Jeopardy!"
            if game_round == 2:
                print('Second Round')
                r = "Double Jeopardy!"
            else:
                print('First Round')
            match = show[show["round"] == r]
            board, questions, questions_remaining = init_board(match)
            display(HTML(board.to_html()))
            current_player_idx = 0

            while questions_remaining > 0:

                # Select question
                while True:
                    selection = input(f"{player_score[current_player_idx][0]} - Pick your question (Format: Row#,Column#): ")
                    if len(selection) == 3:
                        if selection[0].isdigit() and selection[1] == "," and selection[2].isdigit():
                            if int(selection[0]) < 5 and int(selection[2]) < len(questions.columns):
                                if board.iloc[int(selection[0]), int(selection[2])] != 0:
                                    break
                    print("Please choose an available question and provide selection in valid format.")

                coordinates = selection.split(",")
                int_coordinates = list(map(lambda c: int(c), coordinates))

                question = questions.iloc[int_coordinates[0], int_coordinates[1]]
                value = board.iloc[int_coordinates[0], int_coordinates[1]]

                # Check answer
                response = input(question[0] + ". ")
                print(question[1])
                while True:
                    correct = input("Were you right? (y/n) ")
                    if correct.lower() == "y":
                        print("Nicely done!\n")
                        player_score[current_player_idx][1] += value
                        break
                    if correct.lower() == "n":
                        print("You'll get it next time :)\n")
                        break
                    print("Please write y or n")

                # Display Leaderboard
                for player, score in player_score:
                    print(f'{player}: {score}')

                # Update and Display Board
                board.iloc[int_coordinates[0], int_coordinates[1]] = 0
                questions_remaining -= 1
                display(HTML(board.to_html()))
                
                # Next Player
                if current_player_idx == len(player_score) - 1:
                    current_player_idx = 0
                else:
                    current_player_idx += 1
            
            if game_round == 1:
                print("That's the end of the first round. On to ROUND 2!\n")
            
            game_round += 1
            
        print("That's the game! Here are the final standings...\n")
        for player, score in player_score:
            print(f'{player}: {score}')
        
        print()
        play_again = input("Would you like to play again? (y/n) ")
        if play_again.lower() == "y":
            continue
        if play_again.lower() == "n":
            print("Thanks for playing!")
            break

In [ ]:
def jeopardy():
    print('THIS... IS... JEOPARDY!!!\n')
    ready = input("Ready to play? (y/n) ")
    if ready.lower() == "y":
        play_game()

jeopardy()